# Creates subgrids based on size and orientation of a known polygon feature grid system
## 1. Load shapefile and initialize the variables
### To run this code you have to install ArcGIS Pro 3.0 and Anaconda3 2021.05(Python 3.8.8 64bit)
### And clone the arcgispro-py3 env to conda env using ArcGIS Pro Package Manager
### To develop code I used Visual Studio Code and installed jupyter notebook extenstion

In [55]:
import arcpy
from arcpy import env
import math
env.overwriteOutput = True

# you have to change this directory into your shapefile directory
env.workspace = r"C:\Users\Shinestar\Documents\ArcPy_Fishnet"

# You can change this value to set subgrids width and height
cellSizeWidth = '20'
cellSizeHeight = '20'

# input shapefile name, you have to change into your shapefile name
inpFeatureClass = 'TG506_SubGrids_110922'
# output shapefile name
outFeatureClass = 'Fishnet_Result'

# this is used for spatial reference
templateExtent = inpFeatureClass
# angle field name of origin shape file attribute table
a_field_name = 'Main_Angle'
# temporary feature class in memory
tempFeatureClass = 'in_memory/Temp'
desc = arcpy.Describe(inpFeatureClass)
# get extent of origin shape file
left = desc.extent.XMin
right = desc.extent.XMax
bottom = desc.extent.YMin
top = desc.extent.YMax

## 2. Calculate the angle of the origin feature polygon grid
### A. Create new field 'Main_Angle'


In [56]:
arcpy.AddField_management(inpFeatureClass, a_field_name, field_type='double')

<Result 'C:\\Users\\Shinestar\\Documents\\ArcPy_Fishnet\\TG506_SubGrids_110922'>

### B. Calculate Polygon Main Angle


In [57]:
arcpy.CalculatePolygonMainAngle_cartography(inpFeatureClass, a_field_name)

<Result 'C:\\Users\\Shinestar\\Documents\\ArcPy_Fishnet\\TG506_SubGrids_110922'>

### C. Get origin features angle value

In [58]:
angle_values = [row[0] for row in arcpy.da.SearchCursor(inpFeatureClass, a_field_name)]
print(angle_values[0])

88.17


## 2. Generate new Fishnet
### A. Calculate parameters to generate fishnet


In [59]:
origin_coordinate = str(left) + ' ' + str(bottom)
# this parameter is for setting new fishnet angle from origin angle
y_axis_coordinate = str(math.cos(math.radians(angle_values[0])) * 10 + left) + ' ' + str(math.sin(math.radians(angle_values[0])) * 10 + bottom)
corner_point = str(2 * right - left) + ' ' + str(2 * top - bottom)

### B. Create new Fishnet

In [60]:
arcpy.CreateFishnet_management(tempFeatureClass,origin_coordinate, y_axis_coordinate, cell_width=cellSizeWidth, cell_height=cellSizeHeight, corner_coord=corner_point,template=inpFeatureClass)

<Result 'in_memory\\Temp'>

### C. Transform fishnet to close the origin feature class

In [61]:
# move function
def shift_features(in_features, x_shift=None, y_shift=None):
    with arcpy.da.UpdateCursor(in_features, ['SHAPE@XY']) as cursor:
        for row in cursor:
            cursor.updateRow([[row[0][0] + (x_shift or 0),
                               row[0][1] + (y_shift or 0)]])
    return

x_origin_feature_centroid = (left + right)/2
y_origin_feature_centroid = (bottom + top)/2
fdesc = arcpy.Describe(tempFeatureClass)
x_fishnet_centroid = (fdesc.extent.XMin + fdesc.extent.XMax)/2
y_fishnet_centroid = (fdesc.extent.YMin + fdesc.extent.YMax)/2

shift_features(tempFeatureClass,x_origin_feature_centroid - x_fishnet_centroid, y_origin_feature_centroid - y_fishnet_centroid)

## 3. Clip fishnet with orign feature

In [62]:
arcpy.Clip_analysis(tempFeatureClass, inpFeatureClass,outFeatureClass)

<Result 'C:\\Users\\Shinestar\\Documents\\ArcPy_Fishnet\\Fishnet_Result.shp'>